In [1]:
%pwd

'D:\\android_gan_mal2020'

In [2]:
import numpy as np
import pandas as pd
import pickle
import csv
import os
import torch
from torch_geometric.data import Data
from tqdm import tqdm
import networkx as nx
from glob import glob
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm
from scipy import sparse
from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv, SAGEConv, SGConv
import matplotlib.pyplot as plt
from scipy import sparse
np.random.seed(42)
from sklearn.metrics import confusion_matrix,precision_score,recall_score,f1_score,accuracy_score
from torch_geometric.nn import VGAE
from torch import nn
import torch_geometric
from torch_geometric.data import DataLoader
from sklearn import metrics
import random

In [3]:
def get_graph(mat):
#     mat = sparse.load_npz(path)
#     print(mat.shape)
    mat_coo = mat.tocoo()
    mat_csr = mat_coo.tocsr()
    G_new = nx.Graph(mat_csr)
    return G_new
    

In [4]:
def load_feature(path):
    with open(path, 'rb') as f:
        total_feature = pickle.load(f)
        return total_feature

In [5]:
total_feature= load_feature(r'D:\android_gan_mal2020\code_total_feature')

In [6]:
from __future__ import print_function
import numpy as np

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional,SimpleRNN,GRU,Reshape,TimeDistributed,concatenate,BatchNormalization
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Activation, Flatten, UpSampling1D,MaxPooling1D
# from tensorflow.keras.layers.convolutional import MaxPooling1D
# from tensorflow.keras.layers.advanced_activations import LeakyReLU
# from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import model_from_json
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.backend.tensorflow_backend import set_session
# from tensorflow.keras.backend.tensorflow_backend import clear_session
# from tensorflow.keras.backend.tensorflow_backend import get_session  
from tensorflow.keras import metrics
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import tensorflow as tf
# from tensorflow.keras.models import model_from_json

In [7]:
# load json and create model
json_file = open(r"cnn98-12-nodiag.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
black_box_model = model_from_json(loaded_model_json)
# load weights into new model
black_box_model.load_weights(r"cnn98-12-nodiag.h5")

In [8]:
class GraphSage(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GraphSage, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = SAGEConv(5, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
        self.conv5 = SAGEConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
        x = x.relu()
        x = self.conv5(x, edge_index)
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        emb=x
        # 3. Apply a final classifier
        x = F.dropout(x, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)   
        x = self.lin(x)
        return emb,x

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [10]:
black_box_emb_model = GraphSage(hidden_channels=32)
optimizer_bb = torch.optim.Adam(black_box_emb_model.parameters(), lr=0.01)
criterion_bb = torch.nn.CrossEntropyLoss()
black_box_emb_model.load_state_dict(torch.load(r"GraphSagemal2020full_94acc.pt"))
black_box_emb_model = black_box_emb_model.to(device)

In [11]:
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = SAGEConv(in_channels, out_channels)
        self.conv5 = SAGEConv(out_channels, out_channels)
        self.conv_mu = SAGEConv(out_channels, out_channels)
        self.conv_logstd = SAGEConv(out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv5(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [13]:
vae_generator = VGAE(VariationalGCNEncoder(5,32))
optimizer_vae_generator = torch.optim.Adam(vae_generator.parameters(), lr=0.01)
# vae_generator.load_state_dict(torch.load(r"vgae_mal_final_no_diag.pt"))
vae_generator = vae_generator.to(device)

vgae


In [14]:
class GraphSageDis(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GraphSageDis, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = SAGEConv(5, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
        self.conv5 = SAGEConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
        x = x.relu()
        x = self.conv5(x, edge_index)
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        emb=x
        # 3. Apply a final classifier
        x = F.dropout(x, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        x = self.lin3(x)   
        x = self.lin(x)
        return emb,x

In [15]:
discriminator_model = GraphSageDis(hidden_channels=32)
optimizer_discriminator = torch.optim.Adam(discriminator_model.parameters(), lr=0.01)
criterion_discriminator = torch.nn.CrossEntropyLoss()
discriminator_model = discriminator_model.to(device)

In [16]:
class MyEnsemble(torch.nn.Module):
    def __init__(self, vae_generator, discriminator):
        super(MyEnsemble, self).__init__()
        self.vae_generator = vae_generator
        self.discriminator = discriminator
        
    def forward(self, x, edge_index,batch,full,permission):
        z = self.vae_generator.encode(x,edge_index)
#         print(z)
        adj=torch.exp(-torch.cdist(z,z, p=2.0))
        adj = (adj>0.98).float()*1
#         print(adj)
        #to remove self loop
        generated_edge_index_sparse=torch_geometric.utils.dense_to_sparse(adj)[0]    
         # self loop remove
        generated_edge_index_sparse=torch_geometric.utils.remove_self_loops(generated_edge_index_sparse)[0]
        try:
        # convert back to adjacency matrix to perform and and or operation
            generated_edge_index_dense=torch_geometric.utils.to_dense_adj(generated_edge_index_sparse,max_num_nodes=1000)[0]
        except:
            generated_edge_index_dense=torch_geometric.utils.to_dense_adj(edge_index,max_num_nodes=1000)[0]
        #perform and operation with full graph
        generated_edge_index_valid_links=torch.logical_and(full,generated_edge_index_dense)*1
        #perform or operation with the original edge_index to preserver the malware characteristic
        edge_index_dense=torch_geometric.utils.to_dense_adj(edge_index,max_num_nodes=1000)[0]
        generated_edge_index_final=torch.logical_or(generated_edge_index_valid_links,edge_index_dense)*1
        #cover it into sparse to pass into discriminator
        generated_edge_index_final=torch_geometric.utils.dense_to_sparse(generated_edge_index_final)[0]
        emb,output = self.discriminator(x,generated_edge_index_final,batch)
        return output,z


In [17]:
generator=MyEnsemble(vae_generator, discriminator_model)
generator=generator.to(device)
print(generator)

MyEnsemble(
  (vae_generator): VGAE(
    (encoder): VariationalGCNEncoder(
      (conv1): SAGEConv(5, 32)
      (conv5): SAGEConv(32, 32)
      (conv_mu): SAGEConv(32, 32)
      (conv_logstd): SAGEConv(32, 32)
    )
    (decoder): InnerProductDecoder()
  )
  (discriminator): GraphSageDis(
    (conv1): SAGEConv(5, 32)
    (conv2): SAGEConv(32, 32)
    (conv3): SAGEConv(32, 32)
    (conv4): SAGEConv(32, 32)
    (conv5): SAGEConv(32, 32)
    (lin1): Linear(in_features=32, out_features=32, bias=True)
    (lin2): Linear(in_features=32, out_features=32, bias=True)
    (lin3): Linear(in_features=32, out_features=32, bias=True)
    (lin): Linear(in_features=32, out_features=2, bias=True)
  )
)


In [18]:
# overall_graph_path=r'code_block_id_tr_reduced_tr.npz'
# sparse_overall_graph = sparse.load_npz(overall_graph_path)
# overall_graph_edge_index_coo=sparse_overall_graph.tocoo()
# overall_graph_edge_index = torch.tensor([overall_graph_edge_index_coo.row.tolist(),overall_graph_edge_index_coo.col.tolist()], dtype=torch.long) 
# pos_overall_graph_edge_index = overall_graph_edge_index.to(device)
# pos_overall_graph_edge_index =torch_geometric.utils.remove_self_loops(pos_overall_graph_edge_index)[0]
# pos_overall_adj=torch_geometric.utils.to_dense_adj(pos_overall_graph_edge_index)[0]
# print(pos_overall_adj)

In [19]:
overall_graph=load_feature("overall_edge_list")
overall_graph_edge_index = torch.tensor([overall_graph[0],overall_graph[1]], dtype=torch.long) 
pos_overall_graph_edge_index = overall_graph_edge_index.to(device)
pos_overall_graph_edge_index = torch_geometric.utils.remove_self_loops(pos_overall_graph_edge_index)[0]
pos_overall_adj=torch_geometric.utils.to_dense_adj(pos_overall_graph_edge_index)[0]
print(pos_overall_adj)

tensor([[0., 1., 1.,  ..., 0., 0., 0.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        [1., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 1., 0.]], device='cuda:0')


In [20]:
perm_df=pd.read_csv(r'permission_reduced_mal2020.csv')

In [21]:
perm_df.head(2)

,App,com.josephprince.action.media_notification,at.prefs.tweaks_screen_off,sign.Sign7,com.MrDeveloper.SamsungGalaxyNote5LollipopMOROCCOmat,co.peeksoft.stocks.action.FAQ,com.united.mobile.android.permission.C2D_MESSAGE,com.orionhoroscope.daily.horoscope.permission.C2D_MESSAGE,com.MrDeveloper.SamsungGalaxyNote3MALAYSIA,com.mcafee.intent.action.CONTENT_ACTION,...,ata.squid.common.allies.HireAlliesCommonActivity,com.ninefolders.hd3.activity.setup.ACCOUNT_MANAGER_ENTRY,hungama.activity.AppTour,.activities.SocialMediaLikeActivity,net.dotpicko.dotpict.permission.C2D_MESSAGE,com.google.android.googleapps.permission.GOOGLE_AUTH.finance,com.getinvoicebuilder.android.app.permission.C2D_MESSAGE,mmt.intent.action.FLIGHT_SEARCH_RESULT_IF,com.google.android.gms.games.destination.SHOW_INBOX,Type
0,00070891aea5767d858be0ff82253e6637e274cd5e7315...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000aa1f313ddc02b9e365d5dcca92841eac4f40cb88f59...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
perm_df.iloc[:,1:6852]

,com.josephprince.action.media_notification,at.prefs.tweaks_screen_off,sign.Sign7,com.MrDeveloper.SamsungGalaxyNote5LollipopMOROCCOmat,co.peeksoft.stocks.action.FAQ,com.united.mobile.android.permission.C2D_MESSAGE,com.orionhoroscope.daily.horoscope.permission.C2D_MESSAGE,com.MrDeveloper.SamsungGalaxyNote3MALAYSIA,com.mcafee.intent.action.CONTENT_ACTION,com.MrDeveloper.SamsungGalaxyS6EdgeSAUDIARABIA,...,com.avuscapital.trading212_show_add_instruments_phone,ata.squid.common.allies.HireAlliesCommonActivity,com.ninefolders.hd3.activity.setup.ACCOUNT_MANAGER_ENTRY,hungama.activity.AppTour,.activities.SocialMediaLikeActivity,net.dotpicko.dotpict.permission.C2D_MESSAGE,com.google.android.googleapps.permission.GOOGLE_AUTH.finance,com.getinvoicebuilder.android.app.permission.C2D_MESSAGE,mmt.intent.action.FLIGHT_SEARCH_RESULT_IF,com.google.android.gms.games.destination.SHOW_INBOX
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16711,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16712,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16713,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16714,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#loading dataset malware
class YooChooseDatasetBen(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(YooChooseDatasetBen, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return [r'code_block.dataset']

    def download(self):
        pass
        
    def process(self):
        data_list = []
        edge_index_path=[r"class0"]
        current=0
        for i in edge_index_path: 
            edge_indexes = glob(f'{i}/*.npz')
            print(len(edge_indexes))
            for edge_index_path in edge_indexes:
                current=current+1
                print(current)
                y=[0]
                y = torch.tensor(y)
                try:
                    sparse_arr = sparse.load_npz(edge_index_path)
                    sparse_arr.setdiag(0)
                    sparse_arr.eliminate_zeros()
                    edge_index_processed=sparse_arr
                    edge_index = torch.tensor([edge_index_processed.row.tolist(),edge_index_processed.col.tolist()], dtype=torch.long)
                    x = torch.tensor(total_feature,dtype=torch.float)
                    app_name=os.path.basename(edge_index_path).replace('.npz','')
                    permission_data=perm_df.loc[perm_df['App']==app_name].iloc[:,1:6852].values.tolist()[0]
                    data = Data(x=x, edge_index=edge_index, y=y,permission=permission_data)
                    data_list.append(data)
                except Exception as e:
                    print(e)
                    print(edge_index_path)
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [24]:
custom_dataset_benign= YooChooseDatasetBen(root=r'bening_gan')

In [25]:
torch.manual_seed(12345)
dataset_benign = custom_dataset_benign.shuffle()
train_dataset_benign = dataset_benign[:2586]
test_dataset_benign = dataset_benign[2586:]
print(test_dataset_benign)
print(f'Number of training graphs: {len(train_dataset_benign)}')
print(f'Number of test graphs: {len(test_dataset_benign)}')
print(dataset_benign.num_node_features)

YooChooseDatasetBen(1109)
Number of training graphs: 2586
Number of test graphs: 1109
5


In [26]:
def discriminator_train(loader):
    discriminator_model.train()
    running_loss = 0.0
    correct = 0
    benign_count=0
    malware_count=0
    for data in loader:  # Iterate in batches over the training dataset.
         data.x = data.x.to(device)
         data.edge_index= data.edge_index.to(device)
         data.batch=data.batch.to(device)
         data.y = data.y.to(device)
         emb,out = discriminator_model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
#          print(out)
         pred = out.argmax(dim=1)
#          print(pred.dtype)
#          print(data.y.dtype)
         output=pred.cpu().numpy()
         for i in output:
            if i ==0:
                benign_count+=1
            else:
                malware_count+=1
         
         correct += int((pred == data.y).sum())
#          print(pred[0].dtype)
#          print(data.y[0].dtype)
#          print("prediction",pred)
#          print("data y",data.y)
         loss = criterion_discriminator(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer_discriminator.step()  # Update parameters based on gradients.
         optimizer_discriminator.zero_grad() # Clear gradients.
         running_loss += loss.item() * data.num_graphs
    running_loss /= len(loader.dataset)
    print(benign_count)
    print(malware_count)
    return correct / len(loader.dataset),running_loss

In [28]:
def get_discriminator_prediction(data):
    discriminator_model.eval()
    data=data.to(device)
    out= discriminator_model(data.x, data.edge_index, data.batch)  
    pred = out.argmax(dim=1)
    pred_numpy = pred.cpu().numpy()
    return pred_numpy
    

In [29]:
def mydecoder(edge_index,z,pos_overall_adj):
    adj=torch.exp(-torch.cdist(z,z, p=2.0))
    adj = (adj>0.98).float()*1
#     adj = torch.matmul(z, z.t())
#     adj=torch.sigmoid(adj)
    #to remove self loop
    generated_edge_index_sparse=torch_geometric.utils.dense_to_sparse(adj)[0] 
    
     # self loop remove
    generated_edge_index_sparse=torch_geometric.utils.remove_self_loops(generated_edge_index_sparse)[0]
    # convert back to adjacency matrix to perform and and or operation
    try:
        generated_edge_index_dense=torch_geometric.utils.to_dense_adj(generated_edge_index_sparse,max_num_nodes=1000)[0]
        exception=False
    except:
        generated_edge_index_dense=torch_geometric.utils.to_dense_adj(edge_index,max_num_nodes=1000)[0]
        exception=True
#     print(pos_overall_adj)
    #perform and operation with full graph
    generated_edge_index_valid_links=torch.logical_and(pos_overall_adj,generated_edge_index_dense)*1
    #print(generated_edge_index_valid_links)
    #perform or operation with the original edge_index to preserver the malware characteristic
    edge_index_dense=torch_geometric.utils.to_dense_adj(edge_index,max_num_nodes=1000)[0]
    generated_edge_index_final=torch.logical_or(generated_edge_index_valid_links,edge_index_dense)*1
    #cover it into sparse to pass into discriminator
    generated_edge_index_final=torch_geometric.utils.dense_to_sparse(generated_edge_index_final)[0]
    return generated_edge_index_final,exception
    

In [31]:
def get_black_box_p_emb_prediction(permission_data,embedding):
    pred_soft=black_box_model.predict([np.array(permission_data).reshape(1,6851),embedding.detach().cpu().numpy()])
    if pred_soft[0]<0.5:
        pred=0
    else:
        pred=1
    return torch.tensor([pred])

In [32]:
benign_list=[]
for data in train_dataset_benign:
    data.y= torch.tensor([0])
    benign_list.append(data)

In [33]:
def true_positive_rate_calculator(prediction_list,true_list):
    confusion=confusion_matrix(true_list,prediction_list)
    print(confusion)
    TP=confusion[1][1]
    FN=confusion[1][0]
    true_positive_rate=TP/(TP+FN)
    return true_positive_rate
    
    

In [34]:
def get_embedding(data):
    discriminator_model.eval()
    data=data.to(device)
    emb,out = discriminator_model(data.x, data.edge_index, data.batch)  
    pred = out.argmax(dim=1)
    return pred,emb

In [35]:
def get_embedding_bb(data):
    black_box_emb_model.eval()
    data=data.to(device)
    emb,out = black_box_emb_model(data.x, data.edge_index, data.batch)  
    pred = out.argmax(dim=1)
    return pred,emb

In [36]:
malware_network_perm=[]
edge_index_path=[r"class1"]
current=0
exception_count=0
generated_malware_list=[]
generated_prediction=[]
for i in edge_index_path: 
    edge_indexes = glob(f'{i}/*.npz')
    for edge_index_path in edge_indexes:
        current=current+1
        #get app name
        app_name=os.path.basename(edge_index_path).replace('.npz','')
        #get permission
        permission_data=perm_df.loc[perm_df['App']==app_name].iloc[:,1:6852].values.tolist()[0]
        #load network from data from path
        sparse_arr = sparse.load_npz(edge_index_path)
        #transform intoo coo
        sparse_arr.setdiag(0)
        sparse_arr.eliminate_zeros()
        edge_index_processed=sparse_arr
        #convert in to edge index
        edge_index = torch.tensor([edge_index_processed.row.tolist(),edge_index_processed.col.tolist()], dtype=torch.long)
        malware_network_perm.append((edge_index,permission_data))
        if current==8474:
            break

In [37]:
# print(malware_network_perm[0])

In [ ]:
count=0
true_positive_calcu=[]
#for batching will use data loader to batch and pass on to discriminator
for epoch in range(100):
    print("current iteration",epoch)
    generated_malware_list=[]
    exception_count=0
    generated_prediction=[]
    try:
        current=0
        exception_count=0
        generated_malware_list=[]
        generated_prediction=[]
        for edge_perm in malware_network_perm:
            current=current+1
            #get permission
            permission_data=edge_perm[1]
            edge_index = edge_perm[0]
            #get the x feature
            x = torch.tensor(total_feature,dtype=torch.float)
            random_data = random.choice(benign_list)
            edge_index_0=random_data.edge_index[0].split(int(len(random_data.edge_index[0])/2))[1]
            edge_index_1=random_data.edge_index[1].split(int(len(random_data.edge_index[1])/2))[1]
            benign_noise = torch.empty([2,int(len(random_data.edge_index[1])/2)],dtype=int)
            benign_noise[0]=edge_index_0
            benign_noise[1]=edge_index_1
            noise_edge_index=torch.cat((edge_index,benign_noise),1)
            malware = Data(x=x, edge_index=noise_edge_index)
            malware=malware.to(device)
            z = vae_generator.encode(malware.x,malware.edge_index)
            edge_index,exception=mydecoder(malware.edge_index,z,pos_overall_adj)
            if exception:
                exception_count+=1
            x = torch.tensor(total_feature,dtype=torch.float)
            data = Data(x=x, edge_index=edge_index)
            data.batch=torch.zeros(1000, dtype=torch.long)
            out_graph,emb=get_embedding_bb(data)
            out_perm_graph=get_black_box_p_emb_prediction(permission_data,emb)
            generated_prediction.append(out_perm_graph)
            data.edge_index.cpu()
            data.x=data.x.cpu()
            data.batch=data.batch.cpu()
            data.y=out_perm_graph
            data.batch=None
            generated_malware_list.append(data)
    except Exception as e:
        print(e)
    print("malware generation complete")
    print("graph generation",exception_count)
    pred_list_list=[l.tolist() for l in generated_prediction]
    predlist = [item for sublist in pred_list_list for item in sublist]
    # print(predlist)
    predlist.append(0)
    predlist.append(1)
    true_label_check=[1]*len(predlist)
    print("true positive rate after generation",true_positive_rate_calculator(predlist,true_label_check))
    tpr=true_positive_rate_calculator(predlist,true_label_check)
    if tpr<=0.60:
        print("saved here 0.60")
        torch.save(vae_generator.state_dict(), r"newvgaecnnbb_final_"+str(tpr)+".pt")
    if tpr<=0.50:
        print("saved here 0.50")
        torch.save(vae_generator.state_dict(), r"newvgaecnnbb_final_"+str(tpr)+".pt")
    if tpr<=0.40:
        print("saved here 40")
        torch.save(vae_generator.state_dict(), r"newvgaecnnbb_final_"+str(tpr)+".pt")
    #getting true positive rate after graph generation
    #combine_list=generated_malware_list+benign_list
    benign_train_loader = DataLoader(benign_list, batch_size=64, shuffle=True)
    malware_train_loader= DataLoader(generated_malware_list, batch_size=64, shuffle=True)
    for inner_epoch in range(3):
        train_acc_benign,train_loss_benign = discriminator_train(benign_train_loader)
        print("training benign discriminator")
        print(f'Epoch: {inner_epoch:03d}, discriminator Train Acc: {train_acc_benign:.4f},discriminator Train loss:{train_loss_benign:.4f}')
    for inner_epoch in range(10):
        train_acc_malware,train_loss_malware = discriminator_train(malware_train_loader)
        print("training malware discriminator")
        print(f'Epoch: {inner_epoch:03d}, discriminator Train Acc: {train_acc_malware:.4f},discriminator Train loss:{train_loss_malware:.4f}')
    #---------------------------------------------------------------------------
    # train generator
    for generator_epoch in range(3):
        running_loss = 0.0
        running_loss_dis=0.0
        current=0
        exception_count=0
        generated_malware_list=[]
        generated_prediction=[]
        for edge_perm in malware_network_perm:
            current=current+1
            #get permission
            permission_data=edge_perm[1]
            #load network from data from path
            #convert in to edge index
            edge_index = edge_perm[0]
            #get the x feature
            x = torch.tensor(total_feature,dtype=torch.float)
            random_data = random.choice(benign_list)
            edge_index_0=random_data.edge_index[0].split(int(len(random_data.edge_index[0])/2))[1]
            edge_index_1=random_data.edge_index[1].split(int(len(random_data.edge_index[1])/2))[1]
            benign_noise = torch.empty([2,int(len(random_data.edge_index[1])/2)],dtype=int)
            benign_noise[0]=edge_index_0
            benign_noise[1]=edge_index_1
            noise_edge_index=torch.cat((edge_index,benign_noise),1)
            data = Data(x=x, edge_index=noise_edge_index)
            data.batch=torch.zeros(1000, dtype=torch.long)
            data.y=torch.tensor([0])
            data=data.to(device)
            out,z=generator(data.x,data.edge_index,data.batch,pos_overall_adj,permission_data)
            out=out.to(device)
            loss = vae_generator.recon_loss(z,data.edge_index)
            loss_discriminator = criterion_discriminator(out, data.y)
#             loss=loss_discriminator
            loss = loss+(1 / data.num_nodes) * vae_generator.kl_loss()+loss_discriminator
            loss.backward()  # Derive gradients.
            optimizer_vae_generator.step()  # Update parameters based using gradients.
            optimizer_vae_generator.zero_grad() # Clear gradients.
            running_loss += loss 
            running_loss_dis += loss_discriminator 
            data.batch=None
            data.edge_index=data.edge_index.cpu()
            data.x=data.x.cpu()
            data.y=data.y.cpu()
        print("final_loss")
        running_loss /= len(edge_indexes)
        running_loss_dis /= len(edge_indexes)
        vgae_loss= running_loss - running_loss_dis
        print("loss of vgae",vgae_loss)
        print("loss of generator",running_loss)
        print('loss of discriminator',running_loss_dis)

current iteration 0
malware generation complete
graph generation 8474
[[   0    0]
 [2791 5685]]
true positive rate after generation 0.6707173194903256
[[   0    0]
 [2791 5685]]
2586
0
training benign discriminator
Epoch: 000, discriminator Train Acc: 1.0000,discriminator Train loss:0.0293
2586
0
training benign discriminator
Epoch: 001, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
2586
0
training benign discriminator
Epoch: 002, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
1497
6977
training malware discriminator
Epoch: 000, discriminator Train Acc: 0.6643,discriminator Train loss:38.1604
1708
6766
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.7021,discriminator Train loss:0.5460
1839
6635
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.7075,discriminator Train loss:0.5318
1924
6550
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.7144,discriminator Train loss:0.5176
2124
6350
tra

1794
6680
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.7314,discriminator Train loss:62.2924
1742
6732
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.7324,discriminator Train loss:58.4458
1778
6696
training malware discriminator
Epoch: 007, discriminator Train Acc: 0.7340,discriminator Train loss:57.4021
1762
6712
training malware discriminator
Epoch: 008, discriminator Train Acc: 0.7281,discriminator Train loss:57.4235
1771
6703
training malware discriminator
Epoch: 009, discriminator Train Acc: 0.7285,discriminator Train loss:60.0679
final_loss
loss of vgae tensor(0.1429, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(211.8467, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(211.7038, device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of vgae tensor(0.1413, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(210.7867, device='cuda:0', grad_fn=<DivBackward0>)
loss of discrimina

final_loss
loss of vgae tensor(0.1404, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(537.6956, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(537.5551, device='cuda:0', grad_fn=<DivBackward0>)
current iteration 7
malware generation complete
graph generation 0
[[   0    0]
 [ 295 8181]]
true positive rate after generation 0.9651958470976876
[[   0    0]
 [ 295 8181]]
2522
64
training benign discriminator
Epoch: 000, discriminator Train Acc: 0.9753,discriminator Train loss:19.8368
2586
0
training benign discriminator
Epoch: 001, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
2586
0
training benign discriminator
Epoch: 002, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
8474
0
training malware discriminator
Epoch: 000, discriminator Train Acc: 0.0347,discriminator Train loss:60631.3981
8474
0
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.0347,discriminator Train loss:53761.9769
8474
0
trai

1157
7317
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.7674,discriminator Train loss:158.4759
1148
7326
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.7597,discriminator Train loss:162.4153
1235
7239
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.7587,discriminator Train loss:158.5342
1306
7168
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.7574,discriminator Train loss:155.1058
1263
7211
training malware discriminator
Epoch: 007, discriminator Train Acc: 0.7575,discriminator Train loss:152.1883
1352
7122
training malware discriminator
Epoch: 008, discriminator Train Acc: 0.7503,discriminator Train loss:152.0794
1366
7108
training malware discriminator
Epoch: 009, discriminator Train Acc: 0.7482,discriminator Train loss:151.5659
final_loss
loss of vgae tensor(0.1391, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(538.7765, device='cuda:0', grad_fn=<DivBackward0>)
loss of dis

final_loss
loss of vgae tensor(0.1416, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.1934, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0.0518, device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of vgae tensor(0.1418, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.1418, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
current iteration 14
malware generation complete
graph generation 0
[[   0    0]
 [ 690 7786]]
true positive rate after generation 0.918593676262388
[[   0    0]
 [ 690 7786]]
2586
0
training benign discriminator
Epoch: 000, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
2586
0
training benign discriminator
Epoch: 001, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
2586
0
training benign discriminator
Epoch: 002, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
8458
16
training malware 

...

8474
0
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.1291,discriminator Train loss:16845.5905
8474
0
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.1291,discriminator Train loss:15726.3845
8474
0
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.1291,discriminator Train loss:14646.1091
8474
0
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.1291,discriminator Train loss:13545.4086
8474
0
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.1291,discriminator Train loss:12465.8708
8474
0
training malware discriminator
Epoch: 007, discriminator Train Acc: 0.1291,discriminator Train loss:11370.7986
8474
0
training malware discriminator
Epoch: 008, discriminator Train Acc: 0.1291,discriminator Train loss:10274.7810
8474
0
training malware discriminator
Epoch: 009, discriminator Train Acc: 0.1291,discriminator Train loss:9125.8429
final_loss
loss of vgae tensor(0.1420, device='cuda:0', g

final_loss
loss of vgae tensor(0.1282, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(2443.5354, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(2443.4072, device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of vgae tensor(0.1311, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(2447.1411, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(2447.0100, device='cuda:0', grad_fn=<DivBackward0>)
current iteration 76
malware generation complete
graph generation 0
[[   0    0]
 [ 647 7829]]
true positive rate after generation 0.9236668239735725
[[   0    0]
 [ 647 7829]]
2502
84
training benign discriminator
Epoch: 000, discriminator Train Acc: 0.9675,discriminator Train loss:108.3006
2586
0
training benign discriminator
Epoch: 001, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
2586
0
training benign discriminator
Epoch: 002, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
8474


8474
0
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.1206,discriminator Train loss:5623.1136
8474
0
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.1206,discriminator Train loss:4626.7375
8474
0
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.1206,discriminator Train loss:3698.1320
8474
0
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.1206,discriminator Train loss:2803.9568
8474
0
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.1206,discriminator Train loss:1943.1497
7744
730
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.1850,discriminator Train loss:1105.6997
5635
2839
training malware discriminator
Epoch: 007, discriminator Train Acc: 0.3799,discriminator Train loss:540.0966
3656
4818
training malware discriminator
Epoch: 008, discriminator Train Acc: 0.5494,discriminator Train loss:272.9854
2292
6182
training malware discriminator
Epoch: 009, discr

final_loss
loss of vgae tensor(0.1420, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.1420, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of vgae tensor(0.1409, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.1409, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of vgae tensor(0.1412, device='cuda:0', grad_fn=<SubBackward0>)
loss of generator tensor(0.1412, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(0., device='cuda:0', grad_fn=<DivBackward0>)
current iteration 83
malware generation complete
graph generation 0
[[   0    0]
 [1111 7365]]
true positive rate after generation 0.8689240207645116
[[   0    0]
 [1111 7365]]
2586
0
training benign discriminator
Epoch: 000, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
2586
0
training benign dis

2586
0
training benign discriminator
Epoch: 002, discriminator Train Acc: 1.0000,discriminator Train loss:0.0000
8474
0
training malware discriminator
Epoch: 000, discriminator Train Acc: 0.1872,discriminator Train loss:6535.1611
8474
0
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.1872,discriminator Train loss:5714.8669
8474
0
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.1872,discriminator Train loss:4717.1663
8381
93
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.1937,discriminator Train loss:3659.4810
7628
846
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.2492,discriminator Train loss:2521.1326
6052
2422
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.3644,discriminator Train loss:1706.1481
4880
3594
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.4543,discriminator Train loss:1319.2200
3931
4543
training malware discriminator
Epoch: 007, discri

In [87]:
torch.nn.functional.binary_cross_entropy_with_logits(torch.tensor([0.0]),torch.tensor([0.1]),reduce=False)

tensor([0.6931])

In [33]:
count=0
true_positive_calcu=[]
#for batching will use data loader to batch and pass on to discriminator
for epoch in range(100):
    print("current iteration",epoch)
    generated_malware_list=[]
    exception_count=0
    for malware in train_dataset_malware:
        
        try:
            random_data = random.choice(benign_list)
            edge_index_0=random_data.edge_index[0].split(int(len(random_data.edge_index[0])/2))[0]
            edge_index_1=random_data.edge_index[1].split(int(len(random_data.edge_index[1])/2))[0]
            benign_noise = torch.empty([2,int(len(random_data.edge_index[1])/2)],dtype=int)
            benign_noise[0]=edge_index_0
            benign_noise[1]=edge_index_1
            noise_edge_index=torch.cat((malware.edge_index,benign_noise),1)
            malware.edge_index=noise_edge_index
            count=count+1
            malware=malware.to(device)
            z = vae_generator.encode(malware.x,malware.edge_index)
            edge_index,exception=mydecoder(malware.edge_index,z,pos_overall_adj)
            if exception:
                exception_count+=1
            x = torch.tensor(total_feature,dtype=torch.float)
            data = Data(x=x, edge_index=edge_index)
            data.batch=torch.zeros(7017, dtype=torch.long)
            #get the black_box prediction
            out = get_black_box_prediction(data).cpu()
            data.edge_index=data.edge_index.cpu()
            data.edge_index.cpu()
            data.x=data.x.cpu()
            data.batch=data.batch.cpu()
            data.y=out.cpu()
            out = out.cpu().numpy()
            data.batch=None
            generated_malware_list.append(data)
        except Exception as e:
            print(e)
    print("malware generation complete")
    print("graph generation",exception_count)
    #combine_list=generated_malware_list+benign_list
    benign_train_loader = DataLoader(benign_list, batch_size=64, shuffle=True)
    malware_train_loader= DataLoader(generated_malware_list, batch_size=64, shuffle=True)
    train_acc_benign,train_loss_benign = discriminator_train(benign_train_loader)
    print("training benign discriminator")
    print(f'Epoch: {epoch:03d}, discriminator Train Acc: {train_acc_benign:.4f},discriminator Train loss:{train_loss_benign:.4f}')
    for inner_epoch in range(10):
        train_acc_malware,train_loss_malware = discriminator_train(malware_train_loader)
        print("training malware discriminator")
        print(f'Epoch: {inner_epoch:03d}, discriminator Train Acc: {train_acc_malware:.4f},discriminator Train loss:{train_loss_malware:.4f}')

    #---------------------------------------------------------------------------
    # train generator
    for generator_epoch in range(3):
        running_loss = 0.0
        running_loss_dis=0.0
        for data in train_dataset_malware:
            data.y=torch.tensor([0])
            data.batch=torch.zeros(7017, dtype=torch.long)
            random_data = random.choice(benign_list)
            edge_index_0=random_data.edge_index[0].split(int(len(random_data.edge_index[0])/2))[0]
            edge_index_1=random_data.edge_index[1].split(int(len(random_data.edge_index[1])/2))[0]
            benign_noise = torch.empty([2,int(len(random_data.edge_index[1])/2)],dtype=int)
            benign_noise[0]=edge_index_0
            benign_noise[1]=edge_index_1
            noise_edge_index=torch.cat((data.edge_index,benign_noise),1)
            data.edge_index=noise_edge_index
            data=data.to(device)

            out,z=generator(data.x,data.edge_index,data.batch,pos_overall_adj)
            loss = vae_generator.recon_loss(z,data.edge_index)
            loss_discriminator = criterion_discriminator(out, data.y)
    #        loss=loss_discriminator
    #         print(loss_discriminator)
            loss = loss+(1 / data.num_nodes) * vae_generator.kl_loss()+loss_discriminator
            loss.backward()  # Derive gradients.
            optimizer_vae_generator.step()  # Update parameters based on gradients.
            optimizer_vae_generator.zero_grad() # Clear gradients.
            running_loss += loss 
            running_loss_dis += loss_discriminator 
            data.batch=None
            data.edge_index=data.edge_index.cpu()
            data.x=data.x.cpu()
            data.y=data.y.cpu()
            pred = out.argmax(dim=1)
        print("final_loss")
        running_loss /= len(train_dataset_malware)
        running_loss_dis /= len(train_dataset_malware)
        print("loss of generator",running_loss)
        print('loss of discriminator',running_loss_dis)
    #getting tpr after training generator
    generated_malware_list=[]
    prediction=[]
    exception_count=0
    for malware in train_dataset_malware:
        random_data = random.choice(benign_list)
        edge_index_0=random_data.edge_index[0].split(int(len(random_data.edge_index[0])/2))[0]
        edge_index_1=random_data.edge_index[1].split(int(len(random_data.edge_index[1])/2))[0]
        benign_noise = torch.empty([2,int(len(random_data.edge_index[1])/2)],dtype=int)
        benign_noise[0]=edge_index_0
        benign_noise[1]=edge_index_1
        noise_edge_index=torch.cat((malware.edge_index,benign_noise),1)
        malware.edge_index=noise_edge_index
        malware=malware.to(device)
        z = vae_generator.encode(malware.x,malware.edge_index)
        edge_index,exception=mydecoder(malware.edge_index,z,pos_overall_adj)
        if exception:
            exception_count+=1
        x = torch.tensor(total_feature,dtype=torch.float)
        data = Data(x=x, edge_index=edge_index)
        data.batch=torch.zeros(7017, dtype=torch.long)
        #get the black_box prediction
        out = get_black_box_prediction(data).cpu()
        data.edge_index=data.edge_index.cpu()
        data.x=data.x.cpu()
        data.batch=data.batch.cpu()
        data.y=out.cpu()
        out = out.cpu().numpy()
        prediction.append(out)
        data.batch=None
        generated_malware_list.append(data)
    pred_list_list=[l.tolist() for l in prediction]
    predlist = [item for sublist in pred_list_list for item in sublist]
    predlist.append(0)
    predlist.append(1)
    # print(predlist)
    true_label_check=[1]*len(predlist)
    print("after generator graph generation exception",exception_count)
    tpr=true_positive_rate_calculator(predlist,true_label_check)
    print("true positive rate after training generator",tpr)
    true_positive_calcu.append(true_positive_rate_calculator(predlist,true_label_check))
    if tpr<=0.65:
        print("saved here 0.6")
        torch.save(vae_generator.state_dict(), r"D:\android_gan\vgae_final_"+str(tpr)+".pt")
    if tpr<=0.5:
        print("saved here 0.5")
        torch.save(vae_generator.state_dict(), r"D:\android_gan\vgae_final_"+str(tpr)+".pt")
    if tpr<=0.45:
        print("saved here 0.45")
        torch.save(vae_generator.state_dict(), r"D:\android_gan\vgae_final_"+str(tpr)+".pt")
    
    
    
    

    



current iteration 0
malware generation complete
graph generation 2085
35566
64
training benign discriminator
Epoch: 000, discriminator Train Acc: 0.9982,discriminator Train loss:2.8794
3805
0
training malware discriminator
Epoch: 000, discriminator Train Acc: 0.2707,discriminator Train loss:8155.6756
3805
0
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.2707,discriminator Train loss:7502.5144
3805
0
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.2707,discriminator Train loss:6807.3571
3805
0
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.2707,discriminator Train loss:6158.2030
3805
0
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.2707,discriminator Train loss:5438.7086
3805
0
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.2707,discriminator Train loss:4738.6528
3805
0
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.2707,discriminator Train loss:4016.3

3805
0
training malware discriminator
Epoch: 000, discriminator Train Acc: 0.2689,discriminator Train loss:15744.8595
3805
0
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.2689,discriminator Train loss:14527.8961
3805
0
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.2689,discriminator Train loss:13504.3596
3805
0
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.2689,discriminator Train loss:12339.7873
3805
0
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.2689,discriminator Train loss:11432.3947
3805
0
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.2689,discriminator Train loss:10478.5605
3805
0
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.2689,discriminator Train loss:9641.7064
3805
0
training malware discriminator
Epoch: 007, discriminator Train Acc: 0.2689,discriminator Train loss:8802.4063
3805
0
training malware discriminator
Epoch: 008, discrimi

3805
0
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.2770,discriminator Train loss:15971.8342
3805
0
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.2770,discriminator Train loss:13647.2063
3805
0
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.2770,discriminator Train loss:11660.8409
3805
0
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.2770,discriminator Train loss:9964.9977
3805
0
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.2770,discriminator Train loss:8451.2592
3805
0
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.2770,discriminator Train loss:6983.6692
3801
4
training malware discriminator
Epoch: 007, discriminator Train Acc: 0.2781,discriminator Train loss:5829.0687
3799
6
training malware discriminator
Epoch: 008, discriminator Train Acc: 0.2781,discriminator Train loss:4761.1201
3776
29
training malware discriminator
Epoch: 009, discrimina

3805
0
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.2647,discriminator Train loss:4921.4956
3805
0
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.2647,discriminator Train loss:4248.0401
3805
0
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.2647,discriminator Train loss:3607.2067
3805
0
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.2647,discriminator Train loss:3086.7527
3805
0
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.2647,discriminator Train loss:2628.2961
3805
0
training malware discriminator
Epoch: 007, discriminator Train Acc: 0.2647,discriminator Train loss:2216.5860
3805
0
training malware discriminator
Epoch: 008, discriminator Train Acc: 0.2647,discriminator Train loss:1858.0805
3805
0
training malware discriminator
Epoch: 009, discriminator Train Acc: 0.2647,discriminator Train loss:1513.9162
final_loss
loss of generator tensor(0.1312, device='cuda:0', gra

1117
2688
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.5982,discriminator Train loss:62.7567
1117
2688
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.6003,discriminator Train loss:61.6726
1140
2665
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.5811,discriminator Train loss:62.3128
1139
2666
training malware discriminator
Epoch: 007, discriminator Train Acc: 0.5840,discriminator Train loss:59.6491
1092
2713
training malware discriminator
Epoch: 008, discriminator Train Acc: 0.5995,discriminator Train loss:58.3434
1183
2622
training malware discriminator
Epoch: 009, discriminator Train Acc: 0.5829,discriminator Train loss:57.0171
final_loss
loss of generator tensor(137.5452, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(137.4098, device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of generator tensor(136.4647, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(136.3280,

...

final_loss
loss of generator tensor(250.8830, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(250.7480, device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of generator tensor(258.9796, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(258.8505, device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of generator tensor(256.6163, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(256.4835, device='cuda:0', grad_fn=<DivBackward0>)
after generator graph generation exception 3554
[[   0    0]
 [1058 2749]]
true positive rate after training generator 0.7220908852114526
[[   0    0]
 [1058 2749]]
current iteration 82
malware generation complete
graph generation 3559
35582
48
training benign discriminator
Epoch: 082, discriminator Train Acc: 0.9987,discriminator Train loss:0.4191
3805
0
training malware discriminator
Epoch: 000, discriminator Train Acc: 0.2744,discriminator Train loss:8856.7158
3805
0
training malware discrim

final_loss
loss of generator tensor(2314.0029, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(2313.8826, device='cuda:0', grad_fn=<DivBackward0>)
final_loss
loss of generator tensor(2392.0369, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(2391.9104, device='cuda:0', grad_fn=<DivBackward0>)
after generator graph generation exception 3185
[[   0    0]
 [1032 2775]]
true positive rate after training generator 0.7289204097714737
[[   0    0]
 [1032 2775]]
current iteration 86
malware generation complete
graph generation 3193
35575
55
training benign discriminator
Epoch: 086, discriminator Train Acc: 0.9985,discriminator Train loss:4.9643
3805
0
training malware discriminator
Epoch: 000, discriminator Train Acc: 0.2604,discriminator Train loss:10045.4203
3805
0
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.2604,discriminator Train loss:9698.4065
3805
0
training malware discriminator
Epoch: 002, discriminator Train Acc: 0

final_loss
loss of generator tensor(1532.2299, device='cuda:0', grad_fn=<DivBackward0>)
loss of discriminator tensor(1532.1058, device='cuda:0', grad_fn=<DivBackward0>)
after generator graph generation exception 1557
[[   0    0]
 [1051 2756]]
true positive rate after training generator 0.7239296033622274
[[   0    0]
 [1051 2756]]
current iteration 90
malware generation complete
graph generation 1593
35589
41
training benign discriminator
Epoch: 090, discriminator Train Acc: 0.9988,discriminator Train loss:2.6147
3805
0
training malware discriminator
Epoch: 000, discriminator Train Acc: 0.2728,discriminator Train loss:12283.0616
3805
0
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.2728,discriminator Train loss:11736.7172
3805
0
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.2728,discriminator Train loss:11185.0342
3805
0
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.2728,discriminator Train loss:10618.8319
3805
0
t

after generator graph generation exception 3362
[[   0    0]
 [1057 2750]]
true positive rate after training generator 0.7223535592329918
[[   0    0]
 [1057 2750]]
current iteration 94
malware generation complete
graph generation 3410
35583
47
training benign discriminator
Epoch: 094, discriminator Train Acc: 0.9987,discriminator Train loss:1.2577
3805
0
training malware discriminator
Epoch: 000, discriminator Train Acc: 0.2783,discriminator Train loss:11137.1112
3805
0
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.2783,discriminator Train loss:10864.8637
3805
0
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.2783,discriminator Train loss:10613.3490
3805
0
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.2783,discriminator Train loss:10353.4768
3805
0
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.2783,discriminator Train loss:10074.0152
3805
0
training malware discriminator
Epoch: 005, discrimin

malware generation complete
graph generation 2903
35625
5
training benign discriminator
Epoch: 098, discriminator Train Acc: 0.9999,discriminator Train loss:0.1855
3805
0
training malware discriminator
Epoch: 000, discriminator Train Acc: 0.2812,discriminator Train loss:14670.7030
3805
0
training malware discriminator
Epoch: 001, discriminator Train Acc: 0.2812,discriminator Train loss:14122.4061
3805
0
training malware discriminator
Epoch: 002, discriminator Train Acc: 0.2812,discriminator Train loss:13615.3873
3805
0
training malware discriminator
Epoch: 003, discriminator Train Acc: 0.2812,discriminator Train loss:13101.3294
3805
0
training malware discriminator
Epoch: 004, discriminator Train Acc: 0.2812,discriminator Train loss:12625.1474
3805
0
training malware discriminator
Epoch: 005, discriminator Train Acc: 0.2812,discriminator Train loss:12155.0443
3805
0
training malware discriminator
Epoch: 006, discriminator Train Acc: 0.2812,discriminator Train loss:11702.0538
3805
0
tra